# Download sequences, inspect

In [68]:
import time
from typing import List, Optional
import os
from pathlib import Path

from Bio import Entrez, SeqIO, SeqRecord
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
with open("/Users/prince/ncbi_key.txt", "rt") as f:
    NCBI_API_KEY = f.read().strip()

# Always set your email — NCBI requires this
Entrez.email = "prince.leow@sund.ku.dk"

# Optional: set API key if you have one (for higher rate limits)
Entrez.api_key = NCBI_API_KEY

# Download sequences

## Download top 100 fasta records via Entrez query

In [ ]:
SEARCH_TERM = "Candida albicans[Organism] AND genome"
RETMAX = 100
OUT_DIR = "../data/genomes"
os.makedirs(OUT_DIR, exist_ok=True)

handle = Entrez.esearch(db="nucleotide", term=SEARCH_TERM, retmax=RETMAX)
record = Entrez.read(handle)
handle.close()

id_list = record.get("IdList", [])
print(f"Found {len(id_list)} records.")

if not id_list:
    print("No genome records found.")

for idx, seq_id in enumerate(id_list, start=1):
    print(f"[{idx}/{len(id_list)}] Fetching GenBank ID {seq_id}...")
    try:
        fetch_handle = Entrez.efetch(
            db="nucleotide", id=seq_id, rettype="fasta", retmode="text"
        )
        seq_records = list(SeqIO.parse(fetch_handle, "fasta"))
        fetch_handle.close()
        if not seq_records:
            print(f"  ⚠️  No FASTA found for {seq_id}")
            continue

        # Use accession or id as filename
        filename = os.path.join(OUT_DIR, f"{seq_records[0].id}.fasta")

        # Write to file
        SeqIO.write(seq_records, filename, "fasta")
        print(f"  ✅ Saved {filename}")
    except Exception as e:
        print(f"  ❌ Failed {seq_id}: {e}")

    time.sleep(0.3)  # small delay for NCBI courtesy

print("\nAll done! FASTA files saved to:", os.path.abspath(OUT_DIR))

Found 100 records.
[1/100] Fetching GenBank ID 3064943956...


/Users/prince/miniforge3/envs/monolith/lib/python3.13/site-packages/Bio/SeqIO/FastaIO.py:203: BiopythonDeprecationWarning: Previously, the FASTA parser silently ignored comments at the beginning of the FASTA file (before the first sequence).

Nowadays, the FASTA file format is usually understood not to have any such comments, and most software packages do not allow them. Therefore, the use of comments at the beginning of a FASTA file is now deprecated in Biopython.

In a future Biopython release, this deprecation warning will be replaced by a ValueError. To avoid this, there are three options:

(1) Modify your FASTA file to remove such comments at the beginning of the file.

(2) Use SeqIO.parse with the 'fasta-pearson' format instead of 'fasta'. This format is consistent with the FASTA format defined by William Pearson's FASTA aligner software. Thie format allows for comments before the first sequence; lines starting with the ';' character anywhere in the file are also regarded as comm

  ⚠️  No FASTA found for 3064943956
[2/100] Fetching GenBank ID 3040684979...
  ⚠️  No FASTA found for 3040684979
[3/100] Fetching GenBank ID 2967130401...
  ✅ Saved ../data/genome/pdb|9MQU|A.fasta
[4/100] Fetching GenBank ID 2967130400...
  ✅ Saved ../data/genome/pdb|9MQT|A.fasta
[5/100] Fetching GenBank ID 2967130399...
  ✅ Saved ../data/genome/pdb|9MQS|A.fasta
[6/100] Fetching GenBank ID 2964466316...
  ⚠️  No FASTA found for 2964466316
[7/100] Fetching GenBank ID 2964462378...
  ⚠️  No FASTA found for 2964462378
[8/100] Fetching GenBank ID 2964458267...
  ⚠️  No FASTA found for 2964458267
[9/100] Fetching GenBank ID 2949642251...
  ⚠️  No FASTA found for 2949642251
[10/100] Fetching GenBank ID 2949628994...
  ⚠️  No FASTA found for 2949628994
[11/100] Fetching GenBank ID 2845728557...
  ⚠️  No FASTA found for 2845728557
[12/100] Fetching GenBank ID 2845727158...
  ⚠️  No FASTA found for 2845727158
[13/100] Fetching GenBank ID 2845727136...
  ⚠️  No FASTA found for 2845727136
[14/10

In [74]:
print("Number of records downloaded:")
len(list(Path("../data/genomes").glob("*")))

Number of records downloaded:


59

## Ref genome

Reference genome: https://www.ncbi.nlm.nih.gov/nucleotide/NC_018046.1

In [ ]:
ref_gen_path = "../data/Ref_NC_018046.1.fasta"
record = list(SeqIO.parse(ref_gen_path, format="fasta"))[0]
print("Length of reference genome:")
len(record)

Length of reference genome


33928

# Inspect output of blast ref. genome vs downloaded genomes

In [ ]:
headers = "qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore".split(
    " "
)
df = pd.read_table("../data/results.out", names=headers)
# remove self-hits
df = df[~(df["qseqid"] == df["sseqid"])]

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
30,NC_018046.1,NC_002653.1,99.036,24171,226,5,5681,29848,12053,36219,0.000000e+00,43338.0
31,NC_018046.1,NC_002653.1,99.069,5692,49,4,1,5691,1,5689,0.000000e+00,10214.0
32,NC_018046.1,NC_002653.1,97.809,3606,62,17,29846,33442,9688,6091,0.000000e+00,6205.0
33,NC_018046.1,NC_002653.1,97.809,3606,62,17,29846,33442,36272,39869,0.000000e+00,6205.0
34,NC_018046.1,NC_002653.1,99.092,2642,24,0,27207,29848,12382,9741,0.000000e+00,4747.0
...,...,...,...,...,...,...,...,...,...,...,...,...
167,NC_018046.1,CM084639.1,100.000,28,0,0,5508,5535,5527,5500,3.690000e-04,52.8
168,NC_018046.1,CM084639.1,100.000,28,0,0,22459,22486,28918,28891,3.690000e-04,52.8
169,NC_018046.1,9MQU_A,99.699,332,1,0,2092,2423,1,332,1.740000e-171,608.0
170,NC_018046.1,9MQT_A,99.699,332,1,0,2092,2423,1,332,1.740000e-171,608.0


In [25]:
df.sort_values("length", ascending=False).head(10)

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
30,NC_018046.1,NC_002653.1,99.036,24171,226,5,5681,29848,12053,36219,0.0,43338.0
71,NC_018046.1,JBEPJV010000009.1,98.285,14405,195,23,8197,22563,14605,28995,0.0,25185.0
120,NC_018046.1,CM084639.1,98.285,14405,195,23,8197,22563,14605,28995,0.0,25185.0
121,NC_018046.1,CM084639.1,99.327,7286,49,0,22563,29848,29044,36329,0.0,13184.0
72,NC_018046.1,JBEPJV010000009.1,99.327,7286,49,0,22563,29848,29044,36329,0.0,13184.0
73,NC_018046.1,JBEPJV010000009.1,99.438,5692,20,6,1,5691,1,5681,0.0,10323.0
31,NC_018046.1,NC_002653.1,99.069,5692,49,4,1,5691,1,5689,0.0,10214.0
122,NC_018046.1,CM084639.1,99.438,5692,20,6,1,5691,1,5681,0.0,10323.0
33,NC_018046.1,NC_002653.1,97.809,3606,62,17,29846,33442,36272,39869,0.0,6205.0
32,NC_018046.1,NC_002653.1,97.809,3606,62,17,29846,33442,9688,6091,0.0,6205.0


# Extracting candidate gene from GenBank candidate gene

In [ ]:
gb_file = "../data/Ref_NC_018046.1.gb"
records = [x for x in SeqIO.parse(open(gb_file, "r"), "genbank")][0]
records

In [ ]:
for record in SeqIO.parse(gb_file, "genbank"):
    for feature in record.features:
        if feature.type == "rRNA":
            print(feature)

type: rRNA
location: [17:3147](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:13080327']
    Key: gene, Value: ['rnl']
    Key: locus_tag, Value: ['A568_mgr02']
    Key: product, Value: ['large subunit ribosomal RNA']

type: rRNA
location: [20651:22113](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:13080322']
    Key: gene, Value: ['rns']
    Key: locus_tag, Value: ['A568_mgr01']
    Key: product, Value: ['small subunit ribosomal RNA']



In [ ]:
def extract_feature_from_genbank(
    genbank_path: str,
    gene: Optional[str] = "rnl",
    locus_tag: Optional[str] = "A568_mgr02",
    db_xref: Optional[str] = "GeneID:13080327",
    out_fasta: Optional[str] = None,
) -> List[SeqRecord]:
    """
    Extract features from a single GenBank file that match ANY of:
      /gene == gene
      /locus_tag == locus_tag
      /db_xref contains db_xref

    Parameters
    ----------
    genbank_path : str
        Path to the GenBank (.gb/.gbk) file (may contain one or many records).
    gene : str or None
        Gene name to match (e.g. "rnl"). If None, that criterion is skipped.
    locus_tag : str or None
        Locus tag to match (e.g. "A568_mgr02"). If None, that criterion is skipped.
    db_xref : str or None
        db_xref substring to match (e.g. "GeneID:13080327"). If None, that criterion is skipped.
    out_fasta : str or None
        If provided, write found sequences to this FASTA file. Otherwise only return SeqRecord list.

    Returns
    -------
    List[SeqRecord]
        A list of SeqRecord objects for each matching feature (may be empty).
    """
    matches: List[SeqRecord] = []

    # parse the genbank file (handles multiple records inside a single file)
    for record in SeqIO.parse(genbank_path, "genbank"):
        accession = record.id
        for feat in record.features:
            # only consider features that have qualifiers
            quals = feat.qualifiers if feat.qualifiers is not None else {}

            matched = False
            if gene and "gene" in quals:
                if any(v == gene for v in quals.get("gene", [])):
                    matched = True
            if not matched and locus_tag and "locus_tag" in quals:
                if any(v == locus_tag for v in quals.get("locus_tag", [])):
                    matched = True
            if not matched and db_xref and "db_xref" in quals:
                if any(db_xref == v or db_xref in v for v in quals.get("db_xref", [])):
                    matched = True

            if matched:
                # extract sequence for feature (handles joins/complements)
                seq = feat.extract(record.seq)

                # human-friendly coordinates (1-based inclusive)
                start = int(feat.location.start) + 1
                end = int(feat.location.end)
                strand = feat.location.strand
                strand_sym = "+" if strand == 1 else ("-" if strand == -1 else ".")

                gene_name = quals.get("gene", [""])[0] if quals.get("gene") else ""
                locus = (
                    quals.get("locus_tag", [""])[0] if quals.get("locus_tag") else ""
                )
                dbx = ",".join(quals.get("db_xref", []))

                # build SeqRecord with informative id/description
                seq_id = f"{accession}_{gene_name or locus or start}_{start}_{end}"
                description = f"{accession}|type={feat.type}|gene={gene_name or 'NA'}|locus={locus or 'NA'}|{start}-{end}({strand_sym})"
                if dbx:
                    description += f"|dbx={dbx}"

                rec = SeqRecord(seq, id=seq_id, description=description)
                matches.append(rec)

    # optionally write to fasta
    if out_fasta and matches:
        os.makedirs(os.path.dirname(out_fasta) or ".", exist_ok=True)
        SeqIO.write(matches, out_fasta, "fasta")

    return matches


In [62]:
record = extract_feature_from_genbank(gb_file)
with open("../data/rnl.fsa", "wt") as f:
    SeqIO.write(sequences=record, handle=f, format="fasta")

## Post blast

In [67]:
dfA = pd.read_table("../data/ref_rnl_blast.out", names=headers)
# remove self-hits
dfA = dfA[~(dfA["qseqid"] == dfA["sseqid"])]
dfA.sort_values("length", ascending=False)

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
3,NC_018046.1_rnl_18_3147,NC_002653.1,99.074,3132,26,3,1,3130,18,3148,0.000000e+00,5620
10,NC_018046.1_rnl_18_3147,NC_002653.1,99.074,3132,26,3,1,3130,18,3148,0.000000e+00,5620
1,NC_018046.1_rnl_18_3147,JBEPJV010000009.1,99.553,3131,11,2,1,3130,18,3146,0.000000e+00,5701
2,NC_018046.1_rnl_18_3147,CM084639.1,99.553,3131,11,2,1,3130,18,3146,0.000000e+00,5701
8,NC_018046.1_rnl_18_3147,JBEPJV010000009.1,99.553,3131,11,2,1,3130,18,3146,0.000000e+00,5701
9,NC_018046.1_rnl_18_3147,CM084639.1,99.553,3131,11,2,1,3130,18,3146,0.000000e+00,5701
0,NC_018046.1_rnl_18_3147,NC_018046.1,100.000,3130,0,0,1,3130,18,3147,0.000000e+00,5781
7,NC_018046.1_rnl_18_3147,NC_018046.1,100.000,3130,0,0,1,3130,18,3147,0.000000e+00,5781
4,NC_018046.1_rnl_18_3147,9MQU_A,99.699,332,1,0,2075,2406,1,332,1.600000e-172,608
5,NC_018046.1_rnl_18_3147,9MQT_A,99.699,332,1,0,2075,2406,1,332,1.600000e-172,608
